In [1]:
# %% Package imports
from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, f1_score, accuracy_score
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split, GridSearchCV, KFold, cross_val_score
from deep_audio import Audio, Visualization, Directory, Model, JSON
import numpy as np
from joblib import Parallel, delayed
import multiprocessing
import joblib

In [2]:
kfold_split = False

In [3]:
# %% Load dataset
sampling_rate = 22050

method_algo = 'data'

x, y, mapping = Directory.load_json_data(f'processed/{method_algo}/{method_algo}_{sampling_rate}.json',
                                         inputs_fieldname=method_algo)

In [4]:
# SPLIT 5 FOLDS
# mfcc -> rbf, ovo, 10
# lpc -> poly, ovo, 1000
kernel = 'rbf'
decision_function_shape = 'ovo'
random_state = 42

x_holder = []

for row in x:
    x_holder.append(row.flatten())

x = np.array(x_holder)

n = len(x)

x = x[:n]
y = y[:n]

kf = KFold(n_splits=5, shuffle=True, random_state=random_state)
kf.get_n_splits(x)

5

In [5]:
def process_fold(train_index, test_index):
    x_train, x_val = x[train_index], x[test_index]
    y_train, y_val = y[train_index], y[test_index]
    
    model = svm.SVC(C=10, kernel=kernel, decision_function_shape=decision_function_shape)
        
    model.fit(x_train, y_train)
    
    # TESTA ACCURÁCIAS
    
    score_test = model.score(x_val, y_val)

    score_train = model.score(x_train, y_train)

    y_hat = model.predict(x_val)
    
    # SALVA MODELO
    filename = ''
    
    if kfold_split:
        filename = f'models/svm/5fold/{method_algo}_{sampling_rate}_{kernel}_{decision_function_shape}/acc{score_test}_seed{random_state}.sav'
    else:
        filename = f'models/svm/{method_algo}_{sampling_rate}_{kernel}_{decision_function_shape}/acc{score_test}_seed{random_state}.sav'
    
    Directory.create_directory(filename, file=True)
    
    joblib.dump(model, filename)
    
    # SALVA ACURÁCIAS E PARAMETROS
    
    return {
        'method': 'Support Vector Machines',
        'seed': random_state,
        'feature_method': method_algo,
        'sample_rate': sampling_rate,
        'train_test': [len(x_train), len(x_val)],
        'score_train': score_train,
        'score_test': score_test,
        'f1_micro': f1_score(y_hat, y_val, average='micro'),
        'f1_macro': f1_score(y_hat, y_val, average='macro'),
        'model_file': f'acc{score_test}_seed{random_state}.sav',
        'params': model.get_params()
    }

In [6]:
num_cores = multiprocessing.cpu_count()

if kfold_split:
    dump_info = Parallel(n_jobs=num_cores, verbose=3)(
        delayed(process_fold)(train_index, test_index) for train_index, test_index in kf.split(x))
    JSON.create_json_file(f'models/svm/5fold/{method_algo}_{sampling_rate}_{kernel}_{decision_function_shape}/info.json', dump_info)
else:
    for train_index, test_index in kf.split(x):
        dump_info = process_fold(train_index, test_index)
        break
    JSON.create_json_file(f'models/svm/{method_algo}_{sampling_rate}_{kernel}_{decision_function_shape}/info.json', dump_info)

In [7]:
    dump_info

{'method': 'Support Vector Machines',
 'seed': 42,
 'feature_method': 'data',
 'sample_rate': 22050,
 'train_test': [14169, 3543],
 'score_train': 0.9999294233890889,
 'score_test': 0.9489133502681344,
 'f1_micro': 0.9489133502681344,
 'f1_macro': 0.9452871929969652,
 'model_file': 'acc0.9489133502681344_seed42.sav',
 'params': {'C': 10,
  'break_ties': False,
  'cache_size': 200,
  'class_weight': None,
  'coef0': 0.0,
  'decision_function_shape': 'ovo',
  'degree': 3,
  'gamma': 'scale',
  'kernel': 'rbf',
  'max_iter': -1,
  'probability': False,
  'random_state': None,
  'shrinking': True,
  'tol': 0.001,
  'verbose': False}}